# Deploy to Triton Inference Server locally

description: (preview) deploy an image classification model trained on densenet locally via Triton

Please note that this Public Preview release is subject to the [Supplemental Terms of Use for Microsoft Azure Previews](https://azure.microsoft.com/support/legal/preview-supplemental-terms/).

In [ ]:
!pip install nvidia-pyindex
!pip install --upgrade tritonclient

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

## Download model

It's important that your model have this directory structure for Triton Inference Server to be able to load it. [Read more about the directory structure that Triton expects](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html).

In [1]:
import os
import sys
from pathlib import Path
from src.model_utils import download_triton_models, delete_triton_models

prefix = Path(".")
download_triton_models(prefix)

successfully downloaded model: densenet_onnx
successfully downloaded model: bidaf-9


## Register model

A registered model is a logical container stored in the cloud, containing all files located at `model_path`, which is associated with a version number and other metadata.

In [ ]:
from azureml.core.model import Model

model_path = "models"

model = Model.register(
    model_path=model_path,
    model_name="densenet-onnx-example",
    tags={"area": "Image classification", "type": "classification"},
    description="Image classification trained on Imagenet Dataset",
    workspace=ws,
)

print(model)

## Deploy webservice

In this case we deploy to the local compute, but for other options, see [our documentation](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where?tabs=azcli).


In [ ]:
from azureml.core.webservice import LocalWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from random import randint

service_name = "triton-densenet-onnx-local" + str(randint(10000, 99999))
my_env = Environment.get(ws, "AzureML-Triton").clone("my-triton")

for pip_package in ["pillow", "numpy"]:
    my_env.python.conda_dependencies.add_pip_package(pip_package)

my_env.environment_variables["WORKER_COUNT"] = "1"

inference_config = InferenceConfig(
    # this entry script is where we dispatch a call to the Triton server
    source_directory="src",
    entry_script="score_densenet.py",
    environment=my_env,
)

config = LocalWebservice.deploy_configuration(port=6789)

service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=config,
    overwrite=True,
)

service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())

In [19]:
!docker build . -t mytriton

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/2)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 210B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
[+] Building 0.3s (5/6)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 210B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for nvcr.io/nvidia/tritonserver:20.11-py3     0.0s
 => [1/2] FROM nvcr.io/nvidia/tritons

In [13]:
!docker run -d --rm -p8000:8000 -p8001:8001 -p8002:8002 -v/home/gopalv/azureml-examples/experimental/deploy-triton/models/triton:/models --env AZUREML_MODEL_DIR=/models mytriton

5ee862feaa703fa6f01483a765ac25e105c9195308657f8c29f121138d235769


## Test the webservice

In [14]:
import numpy as np
import requests
import io
from PIL import Image

import tritonclient.http as tritonhttpclient

headers = {"Content-Type": "application/octet-stream"}

test_sample = requests.get("https://aka.ms/peacock-pic", allow_redirects=True).content
#img = Image.open(io.BytesIO(test_sample))
test=np.array([test_sample], dtype=bytes)
test = np.stack(test, axis=0)
input = tritonhttpclient.InferInput('img_in_bytes', test.shape, 'BYTES')
input.set_data_from_numpy(test)
inputs = [input]
client = tritonhttpclient.InferenceServerClient("localhost:8000")
outputs = [tritonhttpclient.InferRequestedOutput('data_0')]
result = client.infer(model_name='densenet_preprocess', inputs=inputs, request_id='1', outputs=outputs)
#resp = requests.post(service.scoring_uri, data=test_sample, headers=headers)
#print(resp.text)

## Delete the webservice and the downloaded model

In [16]:
print(result.get_response())

{'id': '1', 'model_name': 'densenet_preprocess', 'model_version': '1', 'outputs': [{'name': 'data_0', 'datatype': 'BYTES', 'shape': [3, 224, 224], 'parameters': {'binary_data_size': 1919865}}]}


In [ ]:
service.delete()
delete_triton_models(prefix)

# Next steps

Try changing the deployment configuration to [deploy to Azure Kubernetes Service](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-azure-kubernetes-service?tabs=python) for higher availability and better scalability.